# Sagemaker Script mode usage (DL - Tensorflow)

Script mode is a very useful technique that lets you easily run your existing code in Amazon SageMaker, with very little change in codes. This time, we will tackle the simple Deep Learning problem (MNIST) with Tensorflow

We will use tf.keras here, but this works the same for other frameworks (TensorFlow, MXNet, PyTorch, etc.).

Training a Keras CNN on Fashion-MNIST with TensorFlow 2.0

Download the dataset --> [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist/)

In [2]:
import urllib.request
import time
from time import gmtime, strftime
import re
import os
import numpy as np
import random
import boto3
import matplotlib.pyplot as plt
%matplotlib inline

import boto3
import sagemaker

prefix = 'keras-fashion-mnist'
sess = sagemaker.Session()
role = 'arn:aws:iam::570447867175:role/SageMakerNotebookRole' # pass your IAM role name
region = boto3.Session().region_name

print('Sagemaker session :', sess)
print('Prefix :', prefix)
print('Region selected :', region)
print('IAM role :', role)

Sagemaker session : <sagemaker.session.Session object at 0x00000295AC608988>
Prefix : keras-fashion-mnist
Region selected : us-west-2
IAM role : arn:aws:iam::570447867175:role/SageMakerNotebookRole


# 1. Load the dataset

In [3]:
import os
import keras
import numpy as np
import tensorflow as tf

(x_train, y_train), (x_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

Using TensorFlow backend.


# 2. You would need your own __.py file

In [4]:
# Look at the structure of the python file before loading into the Sagemaker Training job
!pygmentize mnist.py

import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Softmax
from keras.optimizers import Adam

import numpy as np
import argparse, os

class FMNISTModel(keras.Model):
    # Create the different layers used by the model
    def __init__(self):
        super(FMNISTModel, self).__init__(name='fmnist_model')
        self.conv2d_1   = Conv2D(64, 3, padding='same', activation='relu',input_shape=(28,28))
        self.conv2d_2   = Conv2D(64, 3, padding='same', activation='relu')
        self.max_pool2d = MaxPooling2D((2, 2), padding='same')
        #self.batch_norm = BatchNormalization()
        self.flatten    = Flatten()
        self.dense1     = Dense(512, activation='relu')
        self.dense2     = Dense(10)
        self.dropout    = Dropout(0.3)
        self.softmax    = Softmax()

    # Chain the layers for forward propagation
    def call(self, x):
        # 1st convolution block
        x = se

#### We have two ways to train the model: Local mode / Sagemaker

# 3. Training in my local PC 

**Here is the important part** /
We are putting "train_instance_type='local'" to utilize my local enviornment (my own laptop) using AWS instance , and with 'train_instance_count=1' because I only have 1 GPU attached

In [6]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist.py',
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='2.0.0', # tensorflow version
                          py_version='py3', # most cases py3
                          script_mode=True,
                          hyperparameters={'epochs': 1} # set hyperparameters
                         )

W0617 22:40:26.894845 21460 local_session.py:391] Windows Support for Local Mode is Experimental


In [ ]:
tf_estimator.fit({'training': 'file://data', 'validation': 'file://data'}, logs = True)

# 3.2. Training in Sagemaker

### 3.2.1 Upload data to S3 bucket

In [8]:
training_input_path   = sess.upload_data('data/training.npz', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('data/validation.npz', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

s3://sagemaker-us-west-2-570447867175/keras-fashion-mnist/training/training.npz
s3://sagemaker-us-west-2-570447867175/keras-fashion-mnist/validation/validation.npz


In [ ]:
sagemaker.Session()

### 3.2.2 Train Start

In [14]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist.py',
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p2.xlarge',
                          framework_version='2.0.0', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 10}
                         )

In [ ]:
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path}) 

# 4. Deploy the trained model

In [ ]:
import time

tf_endpoint_name = 'keras-tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

tf_predictor = tf_estimator.deploy(
                 initial_instance_count=1, 
                 instance_type='ml.m4.xlarge',
                 endpoint_name=tf_endpoint_name)

# 5. Predict with the test set

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = tf_predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

# 6. Close the SageMaker Instance

To make sure we don't get charged after the training/inference is over, we have to delete the endpoint.

In [ ]:
sess.delete_endpoint(endpoint_name=tf_endpoint_name) 